In [8]:
import gradio as gr
import pandas as pd
import os
from PIL import Image as PILImage

def create_chart_viewer_app(excel_path, charts_folder):
    # Load data from Excel
    df = pd.read_excel(excel_path)
    if 'image_id' not in df.columns or 'caption' not in df.columns:
        raise ValueError("Excel file must contain 'image_id' and 'caption' columns")
    current_index = 0
    total_images = len(df)

    def update_display(index):
        if index >= total_images:
            index = 0
        elif index < 0:
            index = total_images - 1
        image_id = str(df.iloc[index]['image_id'])
        caption = str(df.iloc[index]['caption'])
        # Add .png extension if not already present
        if not image_id.lower().endswith('.png'):
            image_filename = f"{image_id}.png"
        else:
            image_filename = image_id
        image_path = os.path.join(charts_folder, image_filename)
        position = f"Image {index + 1} of {total_images}"
        # Check if file exists
        if not os.path.exists(image_path):
            return f"**Image ID:** {image_id}", None, caption, index, position
        return f"**Image ID:** {image_id}", image_path, caption, index, position

    with gr.Blocks() as app:
        gr.Markdown("# Interactive Chart Viewer with Captions")
        index_state = gr.State(value=current_index)

        # IMAGE ID display (Markdown for bold formatting)
        image_id_display = gr.Markdown()

        # Image display
        image_display = gr.Image(label="Chart", type="filepath")

        # Caption display
        caption_display = gr.Textbox(label="Caption", interactive=False)

        # Position indicator
        position_indicator = gr.Textbox(label="Position", interactive=False)

        with gr.Row():
            prev_button = gr.Button("Previous")
            next_button = gr.Button("Next")

        def next_image(index):
            return update_display(index + 1)

        def prev_image(index):
            return update_display(index - 1)

        next_button.click(
            next_image,
            inputs=[index_state],
            outputs=[image_id_display, image_display, caption_display, index_state, position_indicator]
        )
        prev_button.click(
            prev_image,
            inputs=[index_state],
            outputs=[image_id_display, image_display, caption_display, index_state, position_indicator]
        )

        # Initialize with first image
        initial_image_id, initial_image, initial_caption, _, initial_position = update_display(current_index)
        image_id_display.value = initial_image_id
        image_display.value = initial_image
        caption_display.value = initial_caption
        position_indicator.value = initial_position

    app.launch()

# Example usage
if __name__ == "__main__":
    excel_path = "captions.xlsx"
    charts_folder = "Charts"
    create_chart_viewer_app(excel_path, charts_folder)


* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


In [2]:
import gradio as gr
import pandas as pd
import os

def read_image_ids_from_file(file_path):
    with open(file_path, 'r') as f:
        image_ids = [line.strip() for line in f if line.strip()]
    return image_ids

def create_chart_viewer_app(excel_path, charts_folder, ids_txt_path):
    df = pd.read_excel(excel_path)
    image_ids_to_show = read_image_ids_from_file(ids_txt_path)
    filtered_df = df[df['image_id'].astype(str).isin(image_ids_to_show)].reset_index(drop=True)
    total_images = len(filtered_df)

    def update_display(index):
        if total_images == 0:
            return "**Image ID:** None", None, "No images to display.", index, "", None
        if index >= total_images:
            index = 0
        elif index < 0:
            index = total_images - 1
        image_id = str(filtered_df.iloc[index]['image_id'])
        caption = str(filtered_df.iloc[index]['caption'])
        image_filename = f"{image_id}.png" if not image_id.lower().endswith('.png') else image_id
        image_path = os.path.join(charts_folder, image_filename)
        position = f"Image {index + 1} of {total_images}"
        if not os.path.exists(image_path):
            return f"**Image ID:** {image_id}", None, caption, index, position, image_id
        return f"**Image ID:** {image_id}", image_path, caption, index, position, image_id

    def jump_to_image(selected_id):
        # Find the index of the selected image_id
        try:
            idx = filtered_df.index[filtered_df['image_id'].astype(str) == selected_id].tolist()[0]
        except IndexError:
            idx = 0
        return update_display(idx)

    with gr.Blocks() as app:
        gr.Markdown("# Interactive Chart Viewer with Captions (Filtered & Selectable)")
        index_state = gr.State(value=0)
        image_id_display = gr.Markdown()
        image_display = gr.Image(label="Chart", type="filepath")
        caption_display = gr.Textbox(label="Caption", interactive=False)
        position_indicator = gr.Textbox(label="Position", interactive=False)
        # Dropdown for image_id selection
        image_id_selector = gr.Dropdown(
            choices=image_ids_to_show, 
            label="Jump to Image ID",
            interactive=True
        )
        with gr.Row():
            prev_button = gr.Button("Previous")
            next_button = gr.Button("Next")

        def next_image(index):
            return update_display(index + 1)

        def prev_image(index):
            return update_display(index - 1)

        next_button.click(
            next_image,
            inputs=[index_state],
            outputs=[image_id_display, image_display, caption_display, index_state, position_indicator, image_id_selector]
        )
        prev_button.click(
            prev_image,
            inputs=[index_state],
            outputs=[image_id_display, image_display, caption_display, index_state, position_indicator, image_id_selector]
        )
        image_id_selector.change(
            jump_to_image,
            inputs=[image_id_selector],
            outputs=[image_id_display, image_display, caption_display, index_state, position_indicator, image_id_selector]
        )

        # Initialize with first image
        initial_image_id, initial_image, initial_caption, _, initial_position, initial_selector = update_display(0)
        image_id_display.value = initial_image_id
        image_display.value = initial_image
        caption_display.value = initial_caption
        position_indicator.value = initial_position
        image_id_selector.value = initial_selector

    app.launch()

# Example usage
if __name__ == "__main__":
    excel_path = "captions.xlsx"
    charts_folder = "Charts"
    ids_txt_path = "cap.txt"
    create_chart_viewer_app(excel_path, charts_folder, ids_txt_path)


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
